<a href="https://colab.research.google.com/github/Banafshehkh/C.elegans-Control-Backbone/blob/main/Destroying_the_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this level we systematically destroy the network and see if it can reproduce the behavior

In [2]:
import time
import os

import numpy as np
import pandas as pd
import scipy.io as sio
from scipy import integrate, signal, sparse, linalg
from threading import Thread
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation, ArtistAnimation
import seaborn as sns


In [4]:
from google.colab import drive
drive.mount("/content/drive/")


Mounted at /content/drive/


In [6]:

""" Number of Neurons """
N = 279

""" Cell membrane conductance (pS) """
Gc = 0.1

""" Cell Membrane Capacitance """
C = 0.015

""" Gap Junctions (Electrical, 279*279) """
ggap = 1.0
Gg_Static = np.load('/content/drive/MyDrive/connectome/Gg.npy')

""" Synaptic connections (Chemical, 279*279) """
gsyn = 1.0
Gs_Static = np.load('/content/drive/MyDrive/connectome/Gs.npy')

""" Leakage potential (mV) """
Ec = -35.0

""" Directionality (279*1) """
E = np.load('/content/drive/MyDrive/connectome/emask.npy')
E = -48.0 * E
EMat = np.tile(np.reshape(E, N), (N, 1))

""" Synaptic Activity Parameters """
ar = 1.0/1.5 # Synaptic activity's rise time
ad = 5.0/1.5 # Synaptic activity's decay time
B = 0.125 # Width of the sigmoid (mv^-1)

""" Input_Mask/Continuous Transtion """
transit_Mat = np.zeros((2, N))

t_Tracker = 0
Iext = 100000

rate = 0.025
offset = 0.15

t_Switch = 0
transit_End = 10


""" Connectome Arrays """
Gg_Dynamic = Gg_Static.copy()
Gs_Dynamic = Gs_Static.copy()

""" Data matrix stack size """
stack_Size = 5000
init_data_Mat = np.zeros((stack_Size + 5000, N))
data_Mat = np.zeros((stack_Size, N))


InMask = np.zeros(N)
oldMask = np.zeros(N)
newMask = np.zeros(N)

# **Define the Chromosome:** In this case, the chromosome represents the connectivity pattern of the C. elegans network. Each gene in the chromosome corresponds to a connection between two neurons. For example, if you have 279 neurons, the chromosome could be represented as a binary string of length 279*279, where each bit indicates the presence or absence of a connection.

In [7]:
gg_flat = Gg_Dynamic.flatten()
gs_flat = Gs_Dynamic.flatten()


Gg_Dynamic_updated = gg_flat.reshape(Gg_Dynamic.shape)
Gs_Dynamic_updated = gs_flat.reshape(Gs_Dynamic.shape)



Once we have defined the chromosomes representing the electrical and chemical connections, the next step is to integrate them into the genetic algorithm framework. Here's an outline of the next steps:

# **Initialization:** A population of potential solutions is randomly generated. Each solution is represented as a set of parameters or a string of values known as a chromosome.

In [10]:
# each chromosome represents a potential solution for the electrical and chemical connectivity patterns of the C. elegans simulation.

# Calculate the chromosome length based on the connectivity pattern
gg_chromosome_length = np.count_nonzero(Gg_Dynamic)
gs_chromosome_length = np.count_nonzero(Gs_Dynamic)

# Initialize an empty population
population = []
population_size = 50

# Generate random chromosomes for the population
for _ in range(population_size):
    # Create a new chromosome
    gg_chromosome = np.random.choice([0, 1], size=(gg_chromosome_length,))
    gs_chromosome = np.random.choice([0, 1], size=(gs_chromosome_length,))

    # Add the chromosome to the population
    population.append((gg_chromosome, gs_chromosome))


# **Evaluation:** Evaluate the fitness of each chromosome by running the simulation using the corresponding connectivity matrices. The fitness function should measure the performance or desired outcome of the simulation, which could be optimizing specific behaviors or minimizing certain criteria.

The next step after initializing the population is to evaluate the fitness of each chromosome. In your case, the fitness function should be based on the simulation output, which measures the performance or behavior of the C. elegans network.